In [1]:
import os
import re
import h5py
import numpy as np
import pandas as pd

import keras
from keras.utils.io_utils import HDF5Matrix
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, MaxPool2D, Flatten, Dense, Dropout, Activation, Input
from keras.optimizers import SGD
from keras.applications import VGG16
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [2]:
cwd=os.getcwd()
models_filename = os.path.join(cwd,'data/v8_vgg16_model_1.h5')
image_dir = os.path.join(cwd,'data/recipes_splitted/images')
image_size = (224, 224)
batch_size = 1
epochs = 80

In [3]:
image_data_generator = ImageDataGenerator(
rescale = 1./255,
horizontal_flip = False,
fill_mode = "nearest",
zoom_range = 0,
width_shift_range = 0,
height_shift_range=0,
rotation_range=0)


generator_flow = image_data_generator.flow_from_directory(
image_dir,
target_size = (image_size[0], image_size[1]),
batch_size = batch_size, 
class_mode = "categorical",
shuffle=False)


num_of_classes = len(generator_flow.class_indices)
total_number_of_images=generator_flow.n

Found 431 images belonging to 27 classes.


In [4]:
images_label_and_name=generator_flow.filenames
labels=[]
image_names=[]

for i in images_label_and_name:
    splitted_label_and_image_name=re.split('\/',i)
    labels.append(splitted_label_and_image_name[0])
    image_names.append(splitted_label_and_image_name[1])
    
keys=generator_flow.class_indices.keys()
food_categories_sorted=sorted(keys,key=str)

In [5]:
model = VGG16(weights=None, include_top=False, input_shape=(image_size[0], image_size[1], 3))

 
x = model.output
x = Flatten()(x)
x = Dense(101*2, activation="relu", name="dense_1")(x)
x = Dense(101*2, activation="relu", name="dense_2")(x)
predictions = Dense(101, activation="softmax", name="dense_3")(x)
model_final = Model(input=model.input, output=predictions)
model_final.compile(loss="categorical_crossentropy", optimizer='adam', metrics=["accuracy"])
model_final.load_weights(models_filename)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:9: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=Tensor("in...)`
  if __name__ == '__main__':


In [6]:
layer_name = 'dense_2'
intermediate_output_model = Model(inputs=model_final.input,
                                 outputs=model_final.get_layer(layer_name).output)

intermediate_output_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [7]:
#Extracting features from images
features=[]

for n in range(total_number_of_images):
    batch = generator_flow.next()
    image = batch[0][0]
    extracted_features_from_image = intermediate_output_model.predict(np.asarray([image]))
    features.extend(extracted_features_from_image)
    
print("Network extracts",len(features[1]), "dimensionality feature vector from each of",
      len(labels), "images")

Network extracts 202 dimensionality feature vector from each of 431 images


In [8]:
dict_images_extracted_features = {
    'image_names': image_names,
    'labels' : labels,
    'features' : features   
}

df_images_extracted_features=pd.DataFrame(dict_images_extracted_features)


print("Saving extracted features pandas dataframe to: ",
      os.path.join(image_dir,"extracted_features.pkl"))
df_images_extracted_features.to_pickle(os.path.join(image_dir,"extracted_features.pkl"))

print("Extracted features pandas dataframe: \n",df_images_extracted_features)

Saving extracted features pandas dataframe to:  /notebooks/school/image_feature_extraction/data/recipes_splitted/images/extracted_features.pkl
Extracted features pandas dataframe: 
                                               features image_names  \
0    [0.05771321, 0.40469226, 0.17946579, 0.0, 0.0,...     1-1.jpg   
1    [0.24756677, 0.0, 0.81109595, 0.0, 0.0, 0.1121...    10-1.jpg   
2    [0.16576688, 0.0, 0.56071675, 0.0, 0.0, 0.0626...    11-1.jpg   
3    [0.0, 0.0, 0.3406139, 0.0, 0.0, 0.0, 0.4046788...    12-1.jpg   
4    [0.0, 0.09645958, 0.6799826, 0.14097536, 0.0, ...    13-1.jpg   
5    [0.0, 0.0, 0.9172021, 0.0, 0.0, 0.0, 0.0, 0.0,...    14-1.jpg   
6    [0.4587431, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...    15-1.jpg   
7    [0.25609338, 0.64049345, 0.27605394, 0.0, 0.0,...     2-1.jpg   
8    [0.39920503, 0.26843926, 0.5381353, 0.3675958,...     3-1.jpg   
9    [0.49237722, 0.0, 0.0, 0.0, 0.0, 0.7390525, 0....     4-1.jpg   
10   [0.0, 0.36428466, 1.4273645, 0.0, 0.3508249